# PEI.CL Price Analysis

This notebook loads `PEI.CL` from the SQLite database and plots historical price, volume, RSI, and Fibonacci views.

In [ ]:
import sqlite3
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
TICKER = "PEI.CL"
candidate_paths = [Path("stock_data.db"), Path("../stock_data.db")]
db_path = next((p for p in candidate_paths if p.exists()), None)

if db_path is None:
    raise FileNotFoundError("Could not find stock_data.db in current or parent directory.")

db_path

In [ ]:
query = """
SELECT date, open, high, low, close, adj_close, volume, dividends, stock_splits
FROM ticker_prices
WHERE ticker = ?
ORDER BY date
"""

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql_query(query, conn, params=(TICKER,), parse_dates=["date"])

if df.empty:
    raise ValueError(f"No rows found for {TICKER}.")

df.head()

In [ ]:
df["close_20ma"] = df["close"].rolling(20).mean()
df["close_50ma"] = df["close"].rolling(50).mean()
df["close_200ma"] = df["close"].rolling(200).mean()

# Exponential Moving Averages
df["ema_20"] = df["close"].ewm(span=20, adjust=False).mean()
df["ema_50"] = df["close"].ewm(span=50, adjust=False).mean()

# Bollinger Bands (20-day, 2 std)
_bb_std = df["close"].rolling(20).std()
df["bb_upper"] = df["close_20ma"] + 2 * _bb_std
df["bb_lower"] = df["close_20ma"] - 2 * _bb_std


In [ ]:
# Interactive Plotly chart (Price + Volume + MAs)
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 2}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_20ma'], name='20-day MA', line={'width': 1.5, 'dash': 'dot', 'color': 'gold'}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_50ma'], name='50-day MA', line={'width': 1.5, 'color': 'orange'}), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['close_200ma'], name='200-day MA', line={'width': 1.5, 'color': 'red'}), secondary_y=False)

fig.add_trace(go.Bar(x=df['date'], y=df['volume'], name='Volume', opacity=0.25), secondary_y=True)

fig.update_layout(
    title=f'{TICKER} Close Price, Volume & Moving Averages (Plotly)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    bargap=0,
    template='plotly_dark',
)
fig.update_yaxes(title_text='Volume', secondary_y=True)
fig.show()


In [ ]:
# RSI calculation (7, 14 and 21)
delta = df['close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# RSI 7 — fastest, best for short-term signals
avg_gain_7 = gain.ewm(alpha=1/7, min_periods=7, adjust=False).mean()
avg_loss_7 = loss.ewm(alpha=1/7, min_periods=7, adjust=False).mean()
df['rsi_7'] = 100 - (100 / (1 + avg_gain_7 / avg_loss_7))

# RSI 14 — standard medium-term
avg_gain_14 = gain.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
avg_loss_14 = loss.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
df['rsi_14'] = 100 - (100 / (1 + avg_gain_14 / avg_loss_14))

# RSI 21 — slower, better for position/swing confirmation
avg_gain_21 = gain.ewm(alpha=1/21, min_periods=21, adjust=False).mean()
avg_loss_21 = loss.ewm(alpha=1/21, min_periods=21, adjust=False).mean()
df['rsi_21'] = 100 - (100 / (1 + avg_gain_21 / avg_loss_21))


In [ ]:
# Interactive Plotly chart (RSI 7, 14 & 21)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_7'],  name='RSI 7',  line={'color': '#00CC44', 'width': 1.5, 'dash': 'dot'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_14'], name='RSI 14', line={'color': 'orange',  'width': 1.5}))
fig.add_trace(go.Scatter(x=df['date'], y=df['rsi_21'], name='RSI 21', line={'color': '#FF1111', 'width': 1.5, 'dash': 'dash'}))
fig.add_hline(y=70, line_dash='dash', line_color='red',    annotation_text='Overbought 70', annotation_position='left')
fig.add_hline(y=30, line_dash='dash', line_color='#4444FF', annotation_text='Oversold 30',   annotation_position='left')
fig.update_layout(
    title=f'{TICKER} RSI 7, 14 & 21 (Plotly)',
    xaxis_title='Date',
    yaxis_title='RSI',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.update_yaxes(range=[0, 100])
fig.show()


In [ ]:
# Interactive Plotly chart — EMA 20 & EMA 50
# EMAs react faster than simple MAs; useful for short/medium-term signals.
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 1.5, 'color': 'steelblue'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['ema_20'], name='EMA 20', line={'width': 2.5, 'color': '#00CC44'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['ema_50'], name='EMA 50', line={'width': 2.5, 'color': '#FF1111'}))

fig.update_layout(
    title=f'{TICKER} — Exponential Moving Averages (EMA 20 & 50)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.show()


In [ ]:
# Interactive Plotly chart — Bollinger Bands (20-day, ±2 std)
# Upper/Lower bands identify overbought/oversold zones based on volatility.
fig = go.Figure()

# Shaded band (upper → lower fill)
fig.add_trace(go.Scatter(
    x=pd.concat([df['date'], df['date'][::-1]]),
    y=pd.concat([df['bb_upper'], df['bb_lower'][::-1]]),
    fill='toself',
    fillcolor='rgba(180, 180, 255, 0.18)',
    line={'color': 'rgba(0,0,0,0)'},
    name='BB Band',
    showlegend=True,
))

fig.add_trace(go.Scatter(x=df['date'], y=df['bb_upper'], name='BB Upper', line={'width': 2.5, 'color': '#FF1111'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['close_20ma'], name='20-day MA (Mid)', line={'width': 2, 'dash': 'dot', 'color': '#FFD700'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['bb_lower'], name='BB Lower', line={'width': 2.5, 'color': '#00CC44'}))
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line={'width': 1.5, 'color': 'white'}))

fig.update_layout(
    title=f'{TICKER} — Bollinger Bands (20-day, ±2 std)',
    xaxis_title='Date',
    yaxis_title='Price',
    legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'left', 'x': 0},
    template='plotly_dark',
)
fig.show()


## Volume quick wins (2 charts)

Low-hanging volume signals: Relative Volume (RVOL) and On-Balance Volume (OBV).

In [ ]:
# Volume metrics used by the next 2 charts
# Restrict to last 12 months to avoid stale/nonsensical early volume data
cutoff = pd.Timestamp.now() - pd.DateOffset(years=1)
vol_df = df[df["date"] >= cutoff].copy().reset_index(drop=True)

vol = vol_df["volume"].fillna(0).astype(float)
close = vol_df["close"].astype(float)

vol_df["vol_sma_20"] = vol.rolling(20).mean()
vol_df["rvol_20"] = (vol / vol_df["vol_sma_20"]).replace([float("inf"), float("-inf")], pd.NA)

# OBV increments volume when close rises, subtracts when close falls
delta = close.diff().fillna(0)
direction = delta.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
vol_df["obv"] = (direction * vol).cumsum()
vol_df["obv_ema_20"] = vol_df["obv"].ewm(span=20, adjust=False).mean()

print(f"RVOL and OBV columns ready ({len(vol_df)} rows, last 12 months).")

In [ ]:
# Volume chart 1/2: Relative Volume (RVOL 20)
plot_df = vol_df.copy()

fig = go.Figure()
fig.add_trace(go.Bar(x=plot_df["date"], y=plot_df["rvol_20"], name="RVOL 20", marker_color="#00B8D4", opacity=0.7))
fig.add_hline(y=1.0, line_dash="dash", line_color="white", annotation_text="Baseline (1.0)", annotation_position="top right")
fig.add_hline(y=1.5, line_dash="dot", line_color="orange", annotation_text="Expansion (1.5)", annotation_position="top right")

fig.update_layout(
    height=340,
    title=f"{TICKER} Relative Volume (20) — last 12 months",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    showlegend=False
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="RVOL")
fig.show()

In [ ]:
# Volume chart 2/2: OBV + OBV EMA(20)
plot_df = vol_df.copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["obv"], name="OBV", line=dict(color="#80CBC4", width=1.7)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["obv_ema_20"], name="OBV EMA 20", line=dict(color="#FFAB40", width=1.6, dash="dot")))

fig.update_layout(
    height=340,
    title=f"{TICKER} OBV trend confirmation — last 12 months",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11))
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="OBV")
fig.show()

## Breakout framework (volume-agnostic)

This section replaces Fibonacci-style levels with objective breakout diagnostics using Donchian channels, squeeze detection, ADX, MACD, and a composite breakout score.

In [ ]:
from IPython.display import display

# --- Trend, volatility, and momentum inputs for breakout detection ---
high = df["high"].astype(float)
low = df["low"].astype(float)
close = df["close"].astype(float)
prev_close = close.shift(1)

tr = pd.concat([(high - low), (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)

# Donchian channels
df["donchian_20_high"] = high.rolling(20).max()
df["donchian_20_low"] = low.rolling(20).min()
df["donchian_55_high"] = high.rolling(55).max()
df["donchian_55_low"] = low.rolling(55).min()

# Bollinger and Keltner squeeze
df["bb_mid"] = close.rolling(20).mean()
bb_std = close.rolling(20).std()
df["bb_upper"] = df["bb_mid"] + 2.0 * bb_std
df["bb_lower"] = df["bb_mid"] - 2.0 * bb_std

df["atr_20"] = tr.ewm(alpha=1/20, adjust=False, min_periods=20).mean()
df["kc_mid"] = close.ewm(span=20, adjust=False).mean()
df["kc_upper"] = df["kc_mid"] + 1.5 * df["atr_20"]
df["kc_lower"] = df["kc_mid"] - 1.5 * df["atr_20"]
df["squeeze_on"] = (df["bb_upper"] < df["kc_upper"]) & (df["bb_lower"] > df["kc_lower"])

df["bandwidth"] = (df["bb_upper"] - df["bb_lower"]) / df["bb_mid"]
df["bandwidth_pct_252"] = df["bandwidth"].rolling(252, min_periods=30).apply(lambda x: x.rank(pct=True).iloc[-1], raw=False)

# ADX (14) and DI lines
up_move = high.diff()
down_move = low.shift(1) - low
plus_dm = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)

df["atr_14"] = tr.ewm(alpha=1/14, adjust=False, min_periods=14).mean()
plus_dm_smooth = plus_dm.ewm(alpha=1/14, adjust=False, min_periods=14).mean()
minus_dm_smooth = minus_dm.ewm(alpha=1/14, adjust=False, min_periods=14).mean()

df["plus_di_14"] = 100 * (plus_dm_smooth / df["atr_14"])
df["minus_di_14"] = 100 * (minus_dm_smooth / df["atr_14"])
di_sum = (df["plus_di_14"] + df["minus_di_14"]).replace(0, pd.NA)
df["dx_14"] = 100 * (df["plus_di_14"] - df["minus_di_14"]).abs() / di_sum
df["adx_14"] = df["dx_14"].ewm(alpha=1/14, adjust=False, min_periods=14).mean()

# MACD
df["ema_12"] = close.ewm(span=12, adjust=False).mean()
df["ema_26"] = close.ewm(span=26, adjust=False).mean()
df["macd_line"] = df["ema_12"] - df["ema_26"]
df["macd_signal"] = df["macd_line"].ewm(span=9, adjust=False).mean()
df["macd_hist"] = df["macd_line"] - df["macd_signal"]
macd_norm = df["macd_hist"].abs().rolling(100, min_periods=20).max().replace(0, pd.NA)
df["macd_strength"] = (df["macd_hist"] / macd_norm).clip(-1, 1)

# Breakout logic
df["breakout_setup"] = df["bandwidth_pct_252"] < 0.20
df["breakout_price_trigger"] = close > df["donchian_20_high"].shift(1)
df["breakout_adx_confirm"] = (df["adx_14"] > 20) & (df["adx_14"] > df["adx_14"].shift(1))
df["breakout_macd_confirm"] = (df["macd_hist"] > 0) & (df["macd_hist"] > df["macd_hist"].shift(1))
df["breakout_long"] = df["breakout_setup"] & df["breakout_price_trigger"] & df["breakout_adx_confirm"] & df["breakout_macd_confirm"]

# Composite breakout score (0-100)
squeeze_score = ((0.20 - df["bandwidth_pct_252"]).clip(lower=0, upper=0.20) / 0.20) * 30
breakout_distance = (close / df["donchian_20_high"].shift(1) - 1.0).clip(lower=0, upper=0.03)
price_score = (breakout_distance / 0.03) * 30
adx_score = ((df["adx_14"] - 20).clip(lower=0, upper=20) / 20) * 20
macd_score = df["macd_strength"].clip(lower=0) * 20
df["breakout_score"] = (squeeze_score + price_score + adx_score + macd_score).clip(0, 100)

# Market regime labels
if "ema_50" not in df.columns:
    df["ema_50"] = close.ewm(span=50, adjust=False).mean()

df["regime"] = "range"
df.loc[df["squeeze_on"], "regime"] = "squeeze"
df.loc[(df["adx_14"] > 25) & (close >= df["ema_50"]), "regime"] = "uptrend"
df.loc[(df["adx_14"] > 25) & (close < df["ema_50"]), "regime"] = "downtrend"

# Candidate trade levels
df["entry_price"] = df["donchian_20_high"].shift(1)
df["stop_price_atr2"] = df["entry_price"] - 2.0 * df["atr_20"]

print("Breakout indicators ready.")


In [ ]:
# --- Decision tables ---
latest_cols = [
    "date", "close", "regime", "breakout_score", "bandwidth_pct_252",
    "adx_14", "plus_di_14", "minus_di_14", "macd_hist", "squeeze_on",
    "breakout_long", "entry_price", "stop_price_atr2"
]

latest_snapshot = df[latest_cols].tail(1).copy()
latest_snapshot["bandwidth_pct_252"] = (latest_snapshot["bandwidth_pct_252"] * 100).round(1)
latest_snapshot["breakout_score"] = latest_snapshot["breakout_score"].round(1)
latest_snapshot["adx_14"] = latest_snapshot["adx_14"].round(1)
latest_snapshot["plus_di_14"] = latest_snapshot["plus_di_14"].round(1)
latest_snapshot["minus_di_14"] = latest_snapshot["minus_di_14"].round(1)
latest_snapshot["macd_hist"] = latest_snapshot["macd_hist"].round(2)
latest_snapshot["entry_price"] = latest_snapshot["entry_price"].round(2)
latest_snapshot["stop_price_atr2"] = latest_snapshot["stop_price_atr2"].round(2)
latest_snapshot["squeeze_on"] = latest_snapshot["squeeze_on"].map({True: "yes", False: "no"})
latest_snapshot["breakout_long"] = latest_snapshot["breakout_long"].map({True: "yes", False: "no"})
latest_snapshot = latest_snapshot.rename(columns={"bandwidth_pct_252": "bandwidth_pct_252(%)"})

signal_cols = ["date", "close", "breakout_score", "regime", "entry_price", "stop_price_atr2"]
recent_signals = df[df["breakout_long"]][signal_cols].tail(10).copy()
for col in ["close", "breakout_score", "entry_price", "stop_price_atr2"]:
    if col in recent_signals:
        recent_signals[col] = recent_signals[col].round(2)

if recent_signals.empty:
    recent_signals = pd.DataFrame([{"message": "No breakout_long bars yet under current rules."}])

regime_counts = (
    df["regime"]
    .value_counts(dropna=False)
    .rename_axis("regime")
    .reset_index(name="bars")
)
regime_counts["pct"] = (regime_counts["bars"] / regime_counts["bars"].sum() * 100).round(1)

print("Latest snapshot")
display(latest_snapshot)
print("\nRecent breakout bars (max 10)")
display(recent_signals)
print("\nRegime distribution")
display(regime_counts)


In [ ]:
# Breakout chart 1/4: Price + Donchian + signals
plot_df = df.tail(300).copy()
markers = plot_df[plot_df["breakout_long"]]

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["close"], name="Close", line=dict(color="white", width=1.8)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["donchian_20_high"], name="Donchian 20 high", line=dict(color="#00C2FF", width=1.2)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["donchian_20_low"], name="Donchian 20 low", line=dict(color="#00C2FF", width=1.1, dash="dot")))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["donchian_55_high"], name="Donchian 55 high", line=dict(color="#7C4DFF", width=1.1, dash="dash")))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["donchian_55_low"], name="Donchian 55 low", line=dict(color="#7C4DFF", width=1.0, dash="dash")))
fig.add_trace(go.Scatter(x=markers["date"], y=markers["close"], mode="markers", name="Breakout long", marker=dict(color="#00E676", size=10, symbol="triangle-up")))
fig.add_trace(go.Scatter(x=markers["date"], y=markers["stop_price_atr2"], mode="markers", name="ATR stop (2x)", marker=dict(color="#FF5252", size=8, symbol="x")))

fig.update_layout(
    height=520,
    title=f"{TICKER} Price + Donchian + breakout markers",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11)),
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price")
fig.show()


In [ ]:
# Breakout chart 2/4: Volatility compression
plot_df = df.tail(300).copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["bandwidth_pct_252"], name="Bandwidth percentile", line=dict(color="#FFD54F", width=2.0)))
fig.add_trace(
    go.Scatter(
        x=plot_df["date"],
        y=plot_df["squeeze_on"].astype(int) * 0.20,
        mode="lines",
        line=dict(width=0),
        fill="tozeroy",
        fillcolor="rgba(0, 194, 255, 0.22)",
        name="Squeeze on"
    )
)
fig.add_hline(y=0.20, line_dash="dash", line_color="orange", annotation_text="Squeeze threshold", annotation_position="top right")

fig.update_layout(
    height=360,
    title="Volatility compression (Bandwidth percentile)",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11)),
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Pct rank", range=[0, 1])
fig.show()


In [ ]:
# Breakout chart 3/4: Trend strength (ADX / +DI / -DI)
plot_df = df.tail(300).copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["adx_14"], name="ADX 14", line=dict(color="#4FC3F7", width=1.8)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["plus_di_14"], name="+DI 14", line=dict(color="#00E676", width=1.5)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["minus_di_14"], name="-DI 14", line=dict(color="#FF5252", width=1.5)))
fig.add_hline(y=20, line_dash="dash", line_color="orange", annotation_text="ADX confirmation", annotation_position="top right")

fig.update_layout(
    height=360,
    title="Trend strength (ADX / +DI / -DI)",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11)),
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="ADX / DI")
fig.show()


In [ ]:
# Breakout chart 4/4: Momentum (MACD)
plot_df = df.tail(300).copy()

fig = go.Figure()
fig.add_trace(go.Bar(x=plot_df["date"], y=plot_df["macd_hist"], name="MACD hist", marker_color="#90CAF9", opacity=0.55))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["macd_line"], name="MACD line", line=dict(color="#42A5F5", width=1.7)))
fig.add_trace(go.Scatter(x=plot_df["date"], y=plot_df["macd_signal"], name="MACD signal", line=dict(color="#EF5350", width=1.4, dash="dot")))
fig.add_hline(y=0, line_dash="dot", line_color="gray")

fig.update_layout(
    height=380,
    title="Momentum (MACD)",
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white"),
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", y=1.05, x=0.0, font=dict(size=11)),
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="MACD")
fig.show()
